# Predicting Judicial Decisions of the European Court of Human Rights

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from fastai import *
from fastai.text import *
from fastai.utils.mem import gpu_mem_get_free_no_cache
from sklearn.model_selection import train_test_split

In [0]:
import numpy as np
import re
import os
import copy
import torch

In [0]:
def read_dataset(PATH):
    X_dataset = {}
    Y_dataset = {}
    for path, dirs, files in os.walk(PATH):
        for filename in files:
            fullpath = os.path.join(path, filename)
            if "both" not in fullpath:
                with open(fullpath, 'r', encoding="utf8") as file:
                    X_dataset, Y_dataset = add_file_to_dataset(fullpath, X_dataset, Y_dataset, file.read())

    return X_dataset, Y_dataset       

In [0]:
def add_file_to_dataset(fullpath, x_dataset, y_dataset, file):
    article = extract_article(fullpath)
    file = preprocess(file)
    if article not in x_dataset.keys() :
        x_dataset[article] = []
        y_dataset[article] = []
    x_dataset[article] = x_dataset[article] + [file]
    label = 0 if "non-violation" in fullpath else 1
    y_dataset[article] = y_dataset[article] + [label]
    return x_dataset, y_dataset  

We use regex to extract the number of the Article from the fullpath and insert the file into the list under that specific Article.

In [0]:
def extract_article(path): 
    pattern = r"(Article\d+)"
    result = re.search(pattern, path)
    article = result.group(1)
    return article

### Preprocessing 

In [0]:
def preprocess(file): 
    file = extract_paragraphs(file)
    return file

In [0]:
def extract_paragraphs(file): 
    file = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', file)
    pat = r'(PROCEDURE\s*\n.+?)?((THE CIRCUMSTANCES OF THE CASE\s*\n.+?RELEVANT DOMESTIC LAW.+?)|(\n(AS TO THE FACTS|THE FACTS|FACTS)\s*\n.+?))(\nIII\.|THE LAW\s*\n|PROCEEDINGS BEFORE THE COMMISSION\s*\n|ALLEGED VIOLATION OF ARTICLE [0-9]+ OF THE CONVENTION \s*\n|ALLEGED VIOLATION OF ARTICLE .+? OF THE CONVENTION\s*\n)'
    result = re.search(pat, file, re.S |  re.IGNORECASE)
    if result is None:
        pat = r'(PROCEDURE\s*\n.+?)?(\nIII\.|THE LAW\s*\n|PROCEEDINGS BEFORE THE COMMISSION\s*\n|ALLEGED VIOLATION OF ARTICLE [0-9]+ OF THE CONVENTION \s*\n)'
        result = re.search(pat, file, re.S |  re.IGNORECASE)
        content = ""
        if result is not None:
            content += result.group(1)
        else:
            print(file)
    else: 
        content = ""
        if result.group(1) is not None:
            content += result.group(1)
        content += result.group(2)
    return content

### Loading the data

In [0]:
base_path = "/content/drive/My Drive/Colab Notebooks/Datasets/Human rights dataset"

In [0]:
X_train_docs, Y_train_docs = read_dataset(base_path + "/train")
X_extra_test_docs, Y_extra_test_docs = read_dataset(base_path + "/test_violations")

### Combining all the articles according to class

In [0]:
X_train = X_train_docs["Article2"] + X_train_docs["Article3"] + X_train_docs["Article5"] + X_train_docs["Article6"] + X_train_docs["Article8"] + X_train_docs["Article10"] + X_train_docs["Article11"] + X_train_docs["Article13"] + X_train_docs["Article14"]

In [0]:
Y_train = Y_train_docs["Article2"] + Y_train_docs["Article3"] + Y_train_docs["Article5"] + Y_train_docs["Article6"] + Y_train_docs["Article8"] + Y_train_docs["Article10"] + Y_train_docs["Article11"] + Y_train_docs["Article13"] + Y_train_docs["Article14"]

### Creating the Training Classifier DataBunches

Credit to FastAi for explaining and providing the code (https://github.com/fastai/course-nlp/blob/master/5-nn-imdb.ipynb)

In [15]:
base_path = "/content/drive/My Drive/Colab Notebooks"
path = base_path + "/ULMFiTModel"
path

'/content/drive/My Drive/Colab Notebooks/ULMFiTModel'

In [0]:
X_train_np = np.array(X_train)
Y_train_np = np.array(Y_train)

In [0]:
train_cases = {'Case': X_train_np, 'Label': Y_train_np}
all_train_df = DataFrame(train_cases, columns= ['Case', 'Label'])

### Loading the training DataBunches

In [0]:
lm_filename = 'data_lm_export_test20.pkl'
clas_filename = 'data_clas_export_test20.pkl'
clas_test_filename = 'data_clas_test_export_test20.pkl'

In [0]:
base_path = "/content/drive/My Drive/Colab Notebooks"
path = base_path + "/ULMFiTModel"
bs = 16
data_lm = load_data(path, lm_filename, bs=bs)
data_clas = load_data(path, clas_filename, bs=bs)
# data_clas_test = load_data(path, clas_filename, bs=bs)

In [20]:
torch.cuda.set_device(0)
gpu_mem_get_free_no_cache()

11430

### Creating the Test Classifier DataBunches

#### Article 2 databunch


In [0]:
X_test_np = np.array(X_extra_test_docs["Article2"])
Y_test_np = np.array(Y_extra_test_docs["Article2"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [21]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 3 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article3"])
Y_test_np = np.array(Y_extra_test_docs["Article3"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [33]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 5 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article5"])
Y_test_np = np.array(Y_extra_test_docs["Article5"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [39]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 6 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article6"])
Y_test_np = np.array(Y_extra_test_docs["Article6"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [24]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 8 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article8"])
Y_test_np = np.array(Y_extra_test_docs["Article8"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [30]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 10 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article10"])
Y_test_np = np.array(Y_extra_test_docs["Article10"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [36]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 11 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article11"])
Y_test_np = np.array(Y_extra_test_docs["Article11"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [42]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 13 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article13"])
Y_test_np = np.array(Y_extra_test_docs["Article13"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [48]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

#### Article 14 databunch

In [0]:
X_test_np = np.array(X_extra_test_docs["Article14"])
Y_test_np = np.array(Y_extra_test_docs["Article14"])
 
test_cases = {'Case': X_test_np, 'Label': Y_test_np}
test_df = DataFrame(test_cases, columns= ['Case', 'Label'])

In [54]:
data_clas_test = TextClasDataBunch.from_df(path, train_df = all_train_df, valid_df = test_df, vocab=data_lm.vocab, text_cols = 'Case', label_cols = 'Label', bs=16)
data_clas_test.save('data_clas_test_export_test20.pkl')

### Testing

For testing, I am using the classifier trained in notebook 10.1 since the training data remains the same.

In [0]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn_c.load('clas_test20')
data_clas.valid_dl = data_clas_test.valid_dl

In [56]:
y_test_pred, y_true = learn_c.get_preds(ds_type=DatasetType.Valid, ordered=True)

In [0]:
preds = torch.argmax(y_test_pred, dim=1)

#### Article 2 results

In [25]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.18
Accuracy score = 0.8241206030150754
Recall score = 0.8241206030150754
Precision score = 1.0
F1 score = 0.9035812672176309


#### Article 3 results

In [37]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.19
Accuracy score = 0.8131609870740305
Recall score = 0.8131609870740305
Precision score = 1.0
F1 score = 0.896953985742061


#### Article 5 results

In [43]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.25
Accuracy score = 0.7486583184257602
Recall score = 0.7486583184257602
Precision score = 1.0
F1 score = 0.8562659846547315


#### Article 6 results

In [28]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.23
Accuracy score = 0.7744379276637341
Recall score = 0.7744379276637341
Precision score = 1.0
F1 score = 0.8728825230684478


#### Article 8 results

In [34]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.43
Accuracy score = 0.5745967741935484
Recall score = 0.5745967741935484
Precision score = 1.0
F1 score = 0.7298335467349552


#### Article 10 results

In [40]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.45
Accuracy score = 0.5476190476190477
Recall score = 0.5476190476190477
Precision score = 1.0
F1 score = 0.7076923076923077


#### Article 11 results

In [46]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.3
Accuracy score = 0.6966292134831461
Recall score = 0.6966292134831461
Precision score = 1.0
F1 score = 0.8211920529801325


#### Article 13 results

In [52]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.16
Accuracy score = 0.839622641509434
Recall score = 0.839622641509434
Precision score = 1.0
F1 score = 0.9128205128205129


#### Article 14 results

In [61]:
import sklearn.metrics as sm
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_true, preds), 2))
print("Accuracy score =", sm.accuracy_score(y_true, preds))
print("Recall score =", sm.recall_score(y_true, preds))
print("Precision score =", sm.precision_score(y_true, preds))
print("F1 score =", sm.f1_score(y_true, preds))

Logistic regression performance:
Mean absolute error = 0.82
Accuracy score = 0.18181818181818182
Recall score = 0.18181818181818182
Precision score = 1.0
F1 score = 0.3076923076923077


In [60]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
(tn, fp, fn, tp)

(0, 0, 36, 8)